In [ ]:
from IPython.utils import capture
from IPython.display import clear_output
import os

#@markdown ## 구글 드라이브에 ComfyUI 설치 
Google_Install = True

Workspace = 'drive/MyDrive/install_comfyui'

if Google_Install:
  from google.colab import drive
  drive.mount('/content/drive')

print('install...')
if not os.path.isdir(f'/content{Workspace}'):
  with capture.capture_output() as cap:
    !wget https://huggingface.co/ninjaneural/webui/resolve/main/direct/install_comfyui_face.tar -O install_comfyui.tar
    !tar -C /content/drive/MyDrive -xvf install_comfyui.tar
    !rm install_comfyui.tar    
    !sed -i -e "/    init_custom_nodes()/a\    folder_paths.add_model_folder_path('AnimateDiffEvolved_Models', '/content/animatediff/models')\n    folder_paths.add_model_folder_path('AnimateDiffMotion_LoRA', '/content/animatediff/motion_lora')" /content/{Workspace}/main.py
    %cd /content/{Workspace}

print("설치완료. 실행환경 설치을 진행해주세요")

In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## 실행환경 설치

%cd /content

!mkdir /content/models
!mkdir /content/models/checkpoints
!mkdir /content/models/controlnet_models
!mkdir /content/models/upscale_models
!mkdir /content/models/loras
!mkdir /content/models/vae
!mkdir /content/models/clip_vision
!mkdir /content/models/clip

print('install library...')
with capture.capture_output() as cap:
  !apt -y install -qq aria2
  !apt install zstd
  !wget https://huggingface.co/ninjaneural/webui/resolve/main/direct/v1.7.0/lib.zst
  !tar -C / --zstd -xvf lib.zst
  !rm lib.zst

print("설치완료. 모델 다운로드를 진행해주세요")


In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## 모델 다운로드

#@markdown **모델(checkpoint) URL (colab에 다운받는 모델)**
Checkpoint_Url = 'https://civitai.com/api/download/models/245598?type=Model&format=SafeTensor&size=pruned&fp=fp16' #@param {type:"string"}
#@markdown **모델 파일명**
Checkpoint_Filename = 'realisticVision_v60.safetensors' #@param {type:"string"}
#@markdown **모델 다운로드**
Checkpoint_Download = True #@param {type:"boolean"}

#@markdown -----
#@markdown *컨트롤넷 모델*

#@markdown **ControlNet 기본 모델 다운로드**
#@markdown > *openpose, lineart, softedge, depth, canny, tile, ip2p*  
ControlNet = True #@param {type:"boolean"}

#@markdown **ControlNet 추가 모델 다운로드**
#@markdown > *segment, inpaint, scribble, mlsd, normalbae, suffle*  
ControlNet_Extra = False #@param {type:"boolean"}

#@markdown **T2I Adapter 모델 다운로드**
#@markdown > *style, color, sketch, zoedepth, recolor*  
ControlNet_T2I = False #@param {type:"boolean"}

#@markdown -----
#@markdown *커스텀 노드 모델*

#@markdown **AnimateDiff 모델 다운로드**
AnimateDiff_Evolved = True #@param {type:"boolean"}

#@markdown **IPAdapter plus 모델 다운로드**
IPAdapter_plus = False #@param {type:"boolean"}

#@markdown **SVD 모델 다운로드**
SVD_Checkpoint = False #@param {type:"boolean"}

#@markdown **insightface 설치 (reactor 노드사용)**
Insightface = False #@param {type:"boolean"}

print('clip download...')
with capture.capture_output() as cap:

  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors -d /content/models/clip_vision -o model.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/image_encoder/model.safetensors -d /content/models/clip_vision -o sdxl_model.safetensors
  #!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -d /content/models/clip_vision/ -o clip_vit14.bin
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/laion/CLIP-ViT-H-14-laion2B-s32B-b79K/resolve/main/open_clip_pytorch_model.bin -d /content/models/clip -o open_clip_pytorch_model.bin

print('controlnet download...')
with capture.capture_output() as cap:

  ControlNet_Model_Path = "/content/models/controlnet_models"
  if ControlNet:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_openpose_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_lineart_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_softedge_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11f1p_sd15_depth_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_canny_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11f1e_sd15_tile_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11e_sd15_ip2p_fp16.safetensors

  if ControlNet_Extra:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_seg_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_inpaint_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_scribble_fp16.safetensors

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_mlsd_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_normalbae_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11e_sd15_shuffle_fp16.safetensors

  if ControlNet_T2I:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -d {ControlNet_Model_Path} -o t2iadapter_style_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -d {ControlNet_Model_Path} -o t2iadapter_color_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd15v2.pth -d {ControlNet_Model_Path} -o t2iadapter_sketch_sd15v2.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_zoedepth_sd15v1.pth -d .{ControlNet_Model_Path} -o t2iadapter_zoedepth_sd15v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ioclab_sd15_recolor.safetensors -d {ControlNet_Model_Path} -o ioclab_sd15_recolor.safetensors  

print('animatediff download...')
with capture.capture_output() as cap:

  if AnimateDiff_Evolved:
    !mkdir /content/animatediff
    !mkdir /content/animatediff/models
    !mkdir /content/animatediff/motion_lora
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt -d /content/animatediff/models -o mm_sd_v15_v2.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15.ckpt -d /content/animatediff/models -o mm_sd_v15.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v14.ckpt -d /content/animatediff/models -o mm_sd_v14.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v3_sd15_mm.ckpt -d /content/animatediff/motion_lora -o v3_sd15_mm.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalDiff/resolve/main/temporaldiff-v1-animatediff.ckpt -d /content/animatediff/models -o temporaldiff-v1-animatediff.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_PanLeft.ckpt -d /content/animatediff/motion_lora -o v2_lora_PanLeft.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_PanRight.ckpt -d /content/animatediff/motion_lora -o v2_lora_PanRight.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_ZoomOut.ckpt -d /content/animatediff/motion_lora -o v2_lora_ZoomOut.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_ZoomIn.ckpt -d /content/animatediff/motion_lora -o v2_lora_ZoomIn.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_TiltUp.ckpt -d /content/animatediff/motion_lora -o v2_lora_TiltUp.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_TiltDown.ckpt -d /content/animatediff/motion_lora -o v2_lora_TiltDown.ckpt
    
print('ipadapter download...')
with capture.capture_output() as cap:

  if IPAdapter_plus:
    !mkdir /content/ipadapter
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors -d /content/ipadapter -o ip-adapter_sd15.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_light.safetensors -d /content/ipadapter -o ip-adapter_sd15_light.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors -d /content/ipadapter -o ip-adapter-plus_sd15.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors -d /content/ipadapter -o ip-adapter-plus-face_sd15.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-full-face_sd15.safetensors -d /content/ipadapter -o ip-adapter-full-face_sd15.safetensors

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl_vit-h.safetensors -d /content/ipadapter -o ip-adapter_sdxl_vit-h.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors -d /content/ipadapter -o ip-adapter-plus_sdxl_vit-h.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus-face_sdxl_vit-h.safetensors -d /content/ipadapter -o ip-adapter-plus-face_sdxl_vit-h.safetensors

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15.bin -d /content/ipadapter -o ip-adapter-faceid_sd15.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15.bin -d /content/ipadapter -o ip-adapter-faceid-plus_sd15.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin -d /content/ipadapter -o ip-adapter-faceid-plusv2_sd15.bin

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl.bin -d /content/ipadapter -o ip-adapter-faceid_sdxl.bin

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15_lora.safetensors -d /content/models/loras -o ip-adapter-faceid-plus_sd15_lora.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15_lora.safetensors -d /content/models/loras -o ip-adapter-faceid-plusv2_sd15_lora.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15_lora.safetensors -d /content/models/loras -o ip-adapter-faceid_sd15_lora.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl_lora.safetensors -d /content/models/loras -o ip-adapter-faceid_sdxl_lora.safetensors
    
# svd checkpoint
print('svd download...')
with capture.capture_output() as cap:
  if SVD_Checkpoint:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-video-diffusion-img2vid/resolve/main/svd.safetensors -d /content/models/checkpoints -o svd.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-video-diffusion-img2vid-xt/resolve/main/svd_xt.safetensors -d /content/models/checkpoints -o svd_xt.safetensors

  if Insightface:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth -d /content/{Workspace}/models/facerestore_models -o GFPGANv1.4.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ninjaneural/webui/resolve/main/inswapper_128.onnx -d /content/{Workspace}/models/insightface -o inswapper_128.onnx
    !pip install insightface

# checkpoint
print('checkpoint download...')
with capture.capture_output() as cap:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d /content/models/checkpoints -o {Checkpoint_Filename}

  # upscaler
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/models/upscale_models -o 4x-UltraSharp.pth

  # vae
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d /content/models/vae -o vae-ft-mse-840000-ema-pruned.vae.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d /content/models/vae -o kl-f8-anime2.ckpt

  # lora
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/62833?type=Model&format=SafeTensor" -d /content/models/loras -o add_detail.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/latent-consistency/lcm-lora-sdv1-5/resolve/main/pytorch_lora_weights.safetensors" -d /content/models/loras -o lcm-lora-sd15.safetensors


%cd /content/{Workspace}

print("다운로드완료. ComfyUI 실행을 진행해주세요")

In [ ]:
#@markdown ## 추가 커스텀 노드

#@markdown **comfyui_segment_anything**
#@markdown > https://github.com/storyicon/comfyui_segment_anything  
CustomNode_segment_anything = False #@param {type:"boolean"}

if CustomNode_segment_anything:
    !git clone https://github.com/storyicon/comfyui_segment_anything /content/{Workspace}/custom_nodes/comfyui_segment_anything
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://raw.githubusercontent.com/biegert/ComfyUI-CLIPSeg/main/custom_nodes/clipseg.py -d /content/{Workspace}/custom_nodes -o clipseg.py


In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## ComfyUI 실행

#@markdown -----
#@markdown *터널링*

#@markdown **Ngrok**
Ngrok_Key = "" #@param {type:"string"}
#@markdown **Cloudflared**
Cloudflared = False #@param {type:"boolean"}
#@markdown **Localtunnel**
LocalTunnel = True #@param {type:"boolean"}

%cd /content/{Workspace}

if Ngrok_Key:
  !pip install pyngrok
  from pyngrok import conf, ngrok
  conf.get_default().auth_token = Ngrok_Key
  conf.get_default().monitor_thread = False
  ssh_tunnels = ngrok.get_tunnels(conf.get_default())
  if len(ssh_tunnels) == 0:
      ssh_tunnel = ngrok.connect(8188)
      print('address：'+ssh_tunnel.public_url)
  else:
      print('address：'+ssh_tunnels[0].public_url)

if LocalTunnel:
  !npm install -g localtunnel

  import subprocess
  import threading
  import time
  import socket
  import urllib.request

  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()
    print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

    print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
      print(line.decode(), end='')

  threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

if Cloudflared:
  !wget -nc https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
  !dpkg -i cloudflared-linux-amd64.deb

  import subprocess
  import threading
  import time
  import socket
  import urllib.request

  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()
    print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

    p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
      l = line.decode()
      if "trycloudflare.com " in l:
        print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
      #print(l, end='')

  threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()  

!python main.py --dont-print-server